In [ ]:
blend_files=$(ls *.blend)

In [ ]:
for blend_file in *.blend
do
    echo "Auto-selecting best device for $blend_file"
    blender -b "$blend_file" \
    --python-expr "
import bpy
prefs = bpy.context.preferences.addons['cycles'].preferences
prefs.refresh_devices()

# check_device_types OPTIX > CUDA > CPU
device_types = [d.type for d in prefs.devices]
if 'OPTIX' in device_types:
    prefs.compute_device_type = 'OPTIX'
    prefs.refresh_devices()
    [setattr(d, 'use', True) for d in prefs.devices if d.type == 'OPTIX']
    print('✅ Using OPTIX GPU')
elif 'CUDA' in device_types:
    prefs.compute_device_type = 'CUDA'
    prefs.refresh_devices()
    [setattr(d, 'use', True) for d in prefs.devices if d.type == 'CUDA']
    print('✅ Using CUDA GPU')
else:
    prefs.compute_device_type = 'NONE'
    [setattr(d, 'use', True) for d in prefs.devices if d.type == 'CPU']
    print('⚠️ No GPU found, using CPU')

bpy.context.scene.cycles.device = 'GPU'
" \
    -o //"${blend_file}_" -a
done
